In [1]:
import pandas as pd
import geopandas as gpd
import datetime as date

import matplotlib.pyplot as plt
from shapely.geometry import Point

In [2]:
crashes = pd.read_csv('Traffic_Crashes_-_Crashes.csv', low_memory = False)
people = pd.read_csv('Traffic_Crashes_-_People.csv', low_memory = False)

In [3]:
merged = pd.merge(people, crashes, on = "CRASH_RECORD_ID")
merged.head()

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO_x,VEHICLE_ID,CRASH_DATE_x,SEAT_NO,CITY,STATE,ZIPCODE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,9/28/19 3:30,NaN,CHICAGO,IL,60651,...,0.0,0.0,1.0,0.0,3,7,9,41.900043,-87.755577,POINT (-87.755576950444 41.900042872883)
1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,4/13/20 22:50,NaN,CHICAGO,IL,60620,...,0.0,0.0,2.0,0.0,22,2,4,41.736044,-87.653404,POINT (-87.653404241798 41.736044089544)
2,O871920,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827211.0,4/13/20 22:50,NaN,UNKNOWN,NaN,NaN,...,0.0,0.0,2.0,0.0,22,2,4,41.736044,-87.653404,POINT (-87.653404241798 41.736044089544)
3,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/1/15 5:00,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,5,1,11,41.808173,-87.708484,POINT (-87.708483889951 41.808172614433)
4,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/1/15 8:00,NaN,NaN,NaN,NaN,...,0.0,0.0,1.0,0.0,8,1,11,41.981425,-87.659861,POINT (-87.65986141844 41.981425369089)


In [4]:
merged1 = merged.filter(['CRASH_DATE_x', 'ZIPCODE', 'LATITUDE', 'LONGITUDE'])
merged1 = merged1.dropna(thresh = 4)
merged1.shape

(710879, 4)

In [5]:
merged1['CRASH_DATE_x'] = pd.to_datetime(merged1['CRASH_DATE_x']).dt.date

In [13]:
dBefore = date.date(2020, 1, 1)
dAfter = date.date(2022, 1, 1)
dates = merged1[(merged1['CRASH_DATE_x'] > dBefore) & (merged1['CRASH_DATE_x'] < dAfter)]
dates.shape

(237979, 4)

In [22]:
dates.rename(columns = {'ZIPCODE':'zip'}, inplace = True)
dates.head()

/var/folders/2w/g29fpypd3zvcwhkcr6vtfwdr0000gn/T/ipykernel_35942/1785670624.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dates.rename(columns = {'ZIPCODE':'zip'}, inplace = True)


,CRASH_DATE_x,zip,LATITUDE,LONGITUDE
1,2020-04-13,60620,41.736044,-87.653404
18,2020-02-23,60625,41.997662,-87.700128
19,2020-02-23,60645,41.997662,-87.700128
20,2020-04-30,60620,41.851561,-87.710734
21,2020-04-30,60644,41.851561,-87.710734


In [23]:
coords = gpd.read_file('coordinates.geojson')
coords.head()

,objectid,shape_area,shape_len,zip,geometry
0,33,106052287.488,42720.0444058,60647,"MULTIPOLYGON (((-87.67762 41.91776, -87.67761 ..."
1,34,127476050.762,48103.7827213,60639,"MULTIPOLYGON (((-87.72683 41.92265, -87.72693 ..."
2,35,45069038.4783,27288.6096123,60707,"MULTIPOLYGON (((-87.78500 41.90915, -87.78531 ..."
3,36,70853834.3797,42527.9896789,60622,"MULTIPOLYGON (((-87.66707 41.88885, -87.66707 ..."
4,37,99039621.2518,47970.1401531,60651,"MULTIPOLYGON (((-87.70656 41.89555, -87.70672 ..."


In [25]:
# z = pd.merge(dates, coords, how = 'inner', left_on = 'ZIPCODE', right_on='zip')
z = pd.merge(dates, coords, on = "zip")
zipcodes = z.filter(['CRASH_DATE_x', 'zip', 'LATITUDE', 'LONGITUDE'])
zipc

In [ ]:
zipcodes['FREQ'] = 1

In [ ]:
zipcodes = zipcodes.groupby(["zip"]).agg({'LATITUDE': 'mean', 'LONGITUDE': 'mean', 'FREQ': 'sum'})
zipcodes = zipcodes.reset_index()
zipcodes.head()

In [11]:
zipcodes.shape
zipcodes['zip'] = zipcodes['zip'].astype(str)

In [12]:
zipcodes.to_csv('zips.csv', index = False)